# Memory MCP Lab - Testing mcp-memory-libsql API

本 Notebook 用於驗證 `memory_tools.py` 中的 `memory_mcp.session.call_tool()` 使用方式，
以及理解 mcp-memory-libsql 提供的 API 工具，包括：
- `create_entities`: 建立記憶體實體
- `search_nodes`: 搜尋節點
- `read_graph`: 讀取完整圖表
- `delete_entity`: 刪除實體

目標：透過實際測試來驗證 memory_tools.py 中的函數邏輯是否正確。

## Section 1: 環境設置與依賴安裝

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import dotenv_values
import asyncio
import json
from contextlib import AsyncExitStack
from datetime import datetime

# 設定 Python 路徑，以便導入專案模組
project_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(project_root / "backend" / "src"))

# 找到 .env 檔案
env_file = project_root / "backend" / ".env"
if not env_file.exists():
    env_file = Path.home() / "Documents/workspace/CasualTrader/backend/.env"

# 讀取並設定環境變數
if env_file.exists():
    env_vars = dotenv_values(env_file)
    for key, value in env_vars.items():
        os.environ[key] = value.strip('"').strip("'") if value else ""
    print(f"✓ .env 已載入: {env_file}")
else:
    print(f"✗ .env 檔案未找到: {env_file}")

# 驗證必要的環境變數
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"✓ OPENAI_API_KEY: {'已設定' if OPENAI_API_KEY else '未設定'}")


## Section 2: MCP Memory LibSQL 基本概念

MCP (Model Context Protocol) Memory LibSQL 是一個為 AI 模型提供持久化記憶體的 MCP 伺服器。
它使用 LibSQL (SQLite 的輕量級衍生版本) 作為後端存儲。

### 核心特性：
1. **Entity 實體管理**：儲存帶有觀察值的命名實體
2. **關係管理**：定義實體之間的連接
3. **向量搜尋**：支援語義搜尋功能
4. **圖表操作**：完整的知識圖表管理

### 主要工具：
- `create_entities`: 建立或更新實體及其觀察
- `search_nodes`: 基於查詢搜尋相似節點
- `read_graph`: 讀取完整或部分圖表
- `delete_entity`: 刪除實體及其關聯

## Section 3: 初始化 Memory MCP 連接

In [ ]:
from agents.mcp import MCPServerStdio
import logging

# 設置日誌
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# 全域變數用於存儲 MCP 伺服器實例和 exit stack
memory_mcp = None
_exit_stack = None

async def initialize_memory_mcp():
    """
    初始化 Memory MCP 伺服器
    
    Returns:
        MCPServerStdio: Memory MCP 伺服器實例
    """
    global memory_mcp, _exit_stack
    
    try:
        if not _exit_stack:
            _exit_stack = AsyncExitStack()
        
        # 配置 LibSQL 資料庫路徑
        memory_db_path = project_root / "backend" / "memory" / "test_memory.db"
        memory_db_path.parent.mkdir(parents=True, exist_ok=True)
        
        print("📦 初始化 Memory MCP...")
        print(f"   Database: {memory_db_path}")
        
        memory_mcp = await _exit_stack.enter_async_context(
            MCPServerStdio(
                name="memory_mcp",
                params={
                    "command": "npx",
                    "args": ["-y", "mcp-memory-libsql"],
                    "env": {"LIBSQL_URL": f"file:{memory_db_path}"},
                },
                client_session_timeout_seconds=30.0,
            )
        )
        
        print("✓ Memory MCP 初始化完成")
        return memory_mcp
    
    except Exception as e:
        print(f"✗ 初始化失敗: {type(e).__name__}: {str(e)}")
        if _exit_stack:
            await _exit_stack.aclose()
            _exit_stack = None
        raise

async def list_available_tools():
    """
    列出 Memory MCP 提供的所有工具
    """
    global memory_mcp
    
    try:
        if not memory_mcp:
            print("✗ Memory MCP 未初始化")
            return []
        
        # 列出可用工具
        tools_result = await memory_mcp.session.list_tools()
        tools = tools_result.tools if hasattr(tools_result, 'tools') else []
        
        print("\n📋 Memory MCP 可用工具:")
        print("-" * 60)
        for tool in tools:
            print(f"  • {tool.name}")
            if hasattr(tool, 'description'):
                print(f"    描述: {tool.description}")
        
        return tools
    
    except Exception as e:
        print(f"✗ 列出工具失敗: {type(e).__name__}: {str(e)}")
        return []

# 執行初始化
print("=" * 60)
print("正在初始化 Memory MCP...")
print("=" * 60)

memory_mcp = await initialize_memory_mcp()
tools = await list_available_tools()


## Section 4: 測試 create_entities 工具

In [ ]:
async def test_create_entities():
    """
    測試 create_entities 工具
    
    此工具用於建立或更新記憶體中的實體。
    響應格式：包含 content 列表，其中第一項是 TextContent
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 60)
    print("測試: create_entities")
    print("=" * 60)
    
    try:
        # 建立測試實體
        test_entity = {
            "name": "agent_test_123_execution_2025",
            "entityType": "trading_execution",
            "observations": [
                "Mode: AUTO_TRADING",
                "Buy TSMC 2330 100 shares",
            ]
        }
        
        print("\n📤 發送請求:")
        print("   工具: create_entities")
        print(f"   實體: {test_entity['name']}")
        print(f"   類型: {test_entity['entityType']}")
        
        result = await memory_mcp.session.call_tool(
            "create_entities",
            {
                "entities": [test_entity]
            }
        )
        
        # 解析回應
        print(f"\n📥 收到回應:")
        print(f"   類型: {type(result)}")
        print(f"   屬性: {dir(result)}")
        
        if hasattr(result, "content") and result.content:
            print(f"   Content 類型: {type(result.content)}")
            print(f"   Content 長度: {len(result.content)}")
            
            for i, content_item in enumerate(result.content):
                print(f"\n   [Content {i}]")
                print(f"     類型: {type(content_item)}")
                print(f"     屬性: {dir(content_item)}")
                
                if hasattr(content_item, "text"):
                    print(f"     文本: {content_item.text[:200]}...")
                elif hasattr(content_item, "value"):
                    print(f"     值: {content_item.value}")
        else:
            print(f"   無 content 屬性或為空")
        
        print(f"\n✓ create_entities 測試完成")
        return result
    
    except Exception as e:
        print(f"\n✗ 錯誤: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 執行測試
result_create = await test_create_entities()


## Section 5: 測試 search_nodes 工具

## Section 5.5: search_nodes 工作原理詳解

根據官方源碼分析（https://github.com/joleyline/mcp-memory-libsql），`search_nodes` 的查詢邏輯如下：

### 查詢流程

1. **向量搜尋優先**（如果有 embedding）
   - 將文本查詢轉換為 embedding 向量
   - 使用 `vector_distance_cos()` 進行向量相似度搜尋
   - 返回最相似的 5 個實體

2. **回退到文本搜尋**（embedding 生成失敗或搜尋無結果時）
   - 使用 `LIKE` 語句進行模糊匹配
   - 在 `name`、`entity_type`、`observations` 中搜尋
   - 返回最多 5 個結果

3. **重要限制**
   - ⚠️ **最多返回 5 個結果**（不受 limit 參數影響）
   - ⚠️ 向量搜尋只搜尋 `embedding IS NOT NULL` 的實體
   - ⚠️ 文本搜尋是模糊匹配（LIKE %query%）

### 查詢無結果的原因

| 原因 | 描述 |
|-----|------|
| 資料庫為空 | 沒有任何實體被保存 |
| 沒有 embedding | 實體被保存，但沒有向量化信息 |
| 查詢不匹配 | 文本搜尋時，查詢關鍵字不匹配任何實體 |
| 向量相似度過低 | 查詢與任何實體的向量距離都超過閾值 |

## Entity, Observation, Relation 之間的關係

根據官方源碼分析，mcp-memory-libsql 的數據模型採用**知識圖譜（Knowledge Graph）**結構。

### 📊 數據結構圖

```
┌─────────────────────────────────────────────────────────┐
│                    KNOWLEDGE GRAPH                       │
└─────────────────────────────────────────────────────────┘
              
              ┌──────────────┐
              │   ENTITY 1   │
              │ (知識圖的節點) │
              └──────┬───────┘
                     │
         ┌───────────┼───────────┐
         │           │           │
    ┌────v─────┐ ┌──v──────┐ ┌─v────────┐
    │Obs 1     │ │Obs 2    │ │Obs 3     │
    │(觀測/性質)│ │(觀測/性質)│ │(觀測/性質)│
    └──────────┘ └─────────┘ └──────────┘
         │
    ┌────v──────────┐
    │ RELATION      │
    │ (連接關係)     │
    └───┬──────────┘
        │
    ┌───v──────────┐
    │   ENTITY 2   │
    │ (知識圖的節點) │
    └──────────────┘
         │
    ┌────v──────────┐
    │ Obs A, B, C   │
    │  (觀測/性質)   │
    └───────────────┘
```

### 📑 數據庫表結構

根據 `database-service.ts` 的 schema：

#### 1. **ENTITIES 表**（知識圖的節點）
```sql
CREATE TABLE entities (
  name TEXT PRIMARY KEY,          -- 實體唯一名稱 (如 "TSMC_stock")
  entity_type TEXT NOT NULL,      -- 實體類型 (如 "stock", "decision", "analysis")
  embedding F32_BLOB,             -- 向量表示（用於語義搜尋）
  created_at DATETIME             -- 建立時間
)
```

**例子：**
```
name: "trading_decision_2025_01_15"
entity_type: "trading_decision"
embedding: [0.1, 0.2, 0.3, ...]  -- 384維向量
created_at: 2025-01-15 10:30:00
```

#### 2. **OBSERVATIONS 表**（實體的屬性/觀測）
```sql
CREATE TABLE observations (
  id INTEGER PRIMARY KEY,
  entity_name TEXT NOT NULL,      -- 外鍵：指向 entities 表
  content TEXT NOT NULL,          -- 觀測內容
  created_at DATETIME,
  FOREIGN KEY (entity_name) REFERENCES entities(name)
)
```

**例子：**
```
id: 1
entity_name: "trading_decision_2025_01_15"
content: "2025-01-15 10:30 買入決策"

id: 2
entity_name: "trading_decision_2025_01_15"
content: "TSMC 2330 目標價 620 元"

id: 3
entity_name: "trading_decision_2025_01_15"
content: "技術指標: RSI 40, MACD 正向"
```

#### 3. **RELATIONS 表**（實體之間的連接）
```sql
CREATE TABLE relations (
  id INTEGER PRIMARY KEY,
  source TEXT NOT NULL,           -- 外鍵：源實體名稱
  target TEXT NOT NULL,           -- 外鍵：目標實體名稱
  relation_type TEXT NOT NULL,    -- 關係類型 (如 "caused_by", "related_to")
  created_at DATETIME,
  FOREIGN KEY (source) REFERENCES entities(name),
  FOREIGN KEY (target) REFERENCES entities(name)
)
```

**例子：**
```
id: 1
source: "trading_decision_2025_01_15"
target: "market_analysis_2025_01_15"
relation_type: "based_on"

id: 2
source: "trading_decision_2025_01_15"
target: "trading_result_2025_01_14"
relation_type: "follows_up"
```

### 🔗 完整關係示意

```
┌─────────────────────────────────────────────────────────────────┐
│                    ENTITY (實體/節點)                            │
│  name: "trading_decision_2025_01_15"                            │
│  type: "trading_decision"                                       │
│  embedding: [0.1, 0.2, ...]                                     │
└────────┬─────────────────────────────┬─────────────────────────┘
         │                             │
         │ (1:N 關係)                  │ (N:N 關係)
         │                             │
    ┌────v──────────────┐         ┌────v────────────────┐
    │  OBSERVATIONS     │         │   RELATIONS         │
    ├───────────────────┤         ├─────────────────────┤
    │ id: 1             │         │ id: 1               │
    │ content: "買入..."│         │ source: this entity │
    ├───────────────────┤         │ target: other entity│
    │ id: 2             │         │ type: "based_on"    │
    │ content: "目標..."│         └─────────────────────┘
    ├───────────────────┤
    │ id: 3             │
    │ content: "技術..."│
    └───────────────────┘
```

### 💡 三者的角色定義

| 組件 | 用途 | 例子 |
|-----|------|------|
| **Entity** | 知識圖的**節點** | "TSMC股票", "買入決策", "市場分析" |
| **Observation** | Entity 的**屬性或事實** | "TSMC股價 615元", "技術指標看好" |
| **Relation** | Entity 間的**連接** | "決策基於分析", "結果來自決策" |

### 🔍 實際工作流程示例

```python
# 1. 創建第一個實體 + 觀測
await memory_mcp.session.call_tool(
  "create_entities",
  {
    "entities": [{
      "name": "market_analysis_2025_01_15",      # Entity
      "entityType": "market_analysis",
      "observations": [                          # Observations
        "台灣股市開盤 17,500",
        "半導體板塊漲幅 2.3%",
        "TSMC 出現買入機會"
      ]
    }]
  }
)

# 2. 創建第二個實體 + 觀測 + 關係
await memory_mcp.session.call_tool(
  "create_entities",
  {
    "entities": [{
      "name": "trading_decision_2025_01_15",     # Entity
      "entityType": "trading_decision",
      "observations": [                          # Observations
        "基於市場分析決定買入",
        "TSMC 2330 目標價 620 元",
        "買入 1000 股"
      ],
      "relations": [{                            # Relations
        "target": "market_analysis_2025_01_15",
        "relationType": "based_on"
      }]
    }]
  }
)

# 3. 查詢時，會返回完整的關係圖
result = await memory_mcp.session.call_tool("read_graph", {})
# 返回:
# {
#   "nodes": [
#     {entity data},
#     {entity data}
#   ],
#   "relations": [
#     {relation data}
#   ]
# }
```

### 🎯 設計優勢

1. **靈活的屬性系統**
   - Entity 本身只有 name、type、embedding
   - 所有細節信息存在 Observations（可以任意添加）

2. **完整的知識圖譜**
   - 通過 Relations 連接不同實體
   - 支持複雜的業務邏輯表達

3. **高效的向量搜尋**
   - Entity 級別的 embedding 用於語義搜尋
   - Observation 文本用於關鍵詞搜尋

4. **外鍵約束**
   - 保證數據完整性
   - Observations 和 Relations 必須指向有效的 Entity

In [ ]:
async def demo_entity_observation_relation():
    """
    演示 Entity、Observation、Relation 之間的實際關係
    
    使用交易場景展示完整的知識圖譜構建
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 80)
    print("演示: Entity、Observation、Relation 的完整關係")
    print("=" * 80)
    
    # 第一步: 創建市場分析實體（只有觀測，沒有關係）
    print("\n1️⃣  創建市場分析實體（Entity + Observations）")
    print("-" * 80)
    
    market_analysis = {
        "name": "market_analysis_2025_01_15",
        "entityType": "market_analysis",
        "observations": [
            "台灣股市開盤 17,500 點",
            "半導體板塊漲幅 2.3%",
            "外資淨買超 150 億",
            "TSMC 出現技術買入機會"
        ]
    }
    
    print("📝 Entity 信息:")
    print(f"  • name: {market_analysis['name']}")
    print(f"  • entityType: {market_analysis['entityType']}")
    print(f"\n📊 Observations ({len(market_analysis['observations'])} 個):")
    for i, obs in enumerate(market_analysis['observations'], 1):
        print(f"  {i}. {obs}")
    
    try:
        result = await memory_mcp.session.call_tool(
            "create_entities",
            {"entities": [market_analysis]}
        )
        print(f"\n✅ 實體創建成功")
    except Exception as e:
        print(f"❌ 創建失敗: {e}")
        return
    
    # 第二步: 創建交易決策實體（包含 Relation）
    print("\n\n2️⃣  創建交易決策實體（Entity + Observations + Relations）")
    print("-" * 80)
    
    trading_decision = {
        "name": "trading_decision_2025_01_15",
        "entityType": "trading_decision",
        "observations": [
            "時間: 2025-01-15 10:30",
            "決策: 買入 TSMC 2330",
            "數量: 1000 股",
            "目標價: 620 元",
            "技術指標: RSI 35(超賣), MACD 正向交叉",
            "理由: 基於市場分析和技術信號"
        ],
        "relations": [{
            "target": "market_analysis_2025_01_15",
            "relationType": "based_on"  # 此決策基於那個市場分析
        }]
    }
    
    print("📝 Entity 信息:")
    print(f"  • name: {trading_decision['name']}")
    print(f"  • entityType: {trading_decision['entityType']}")
    
    print(f"\n📊 Observations ({len(trading_decision['observations'])} 個):")
    for i, obs in enumerate(trading_decision['observations'], 1):
        print(f"  {i}. {obs}")
    
    print(f"\n🔗 Relations ({len(trading_decision['relations'])} 個):")
    for i, rel in enumerate(trading_decision['relations'], 1):
        print(f"  {i}. 關係類型: {rel['relationType']}")
        print(f"     目標: {rel['target']}")
    
    try:
        result = await memory_mcp.session.call_tool(
            "create_entities",
            {"entities": [trading_decision]}
        )
        print(f"\n✅ 實體創建成功")
    except Exception as e:
        print(f"❌ 創建失敗: {e}")
        return
    
    # 第三步: 創建交易結果實體（有多個relation）
    print("\n\n3️⃣  創建交易結果實體（多個 Relations）")
    print("-" * 80)
    
    trading_result = {
        "name": "trading_result_2025_01_15",
        "entityType": "trading_result",
        "observations": [
            "交易成交時間: 2025-01-15 10:35",
            "成交價: 615 元",
            "成交股數: 1000 股",
            "交易總額: 615,000 元",
            "手續費: 123 元",
            "實際成本: 615,123 元",
            "交易狀態: 成功"
        ],
        "relations": [
            {
                "target": "trading_decision_2025_01_15",
                "relationType": "executes"  # 執行這個決策
            },
            {
                "target": "market_analysis_2025_01_15",
                "relationType": "related_to"  # 與市場分析相關
            }
        ]
    }
    
    print("📝 Entity 信息:")
    print(f"  • name: {trading_result['name']}")
    print(f"  • entityType: {trading_result['entityType']}")
    
    print(f"\n📊 Observations ({len(trading_result['observations'])} 個):")
    for i, obs in enumerate(trading_result['observations'], 1):
        print(f"  {i}. {obs}")
    
    print(f"\n🔗 Relations ({len(trading_result['relations'])} 個):")
    for i, rel in enumerate(trading_result['relations'], 1):
        print(f"  {i}. 關係類型: {rel['relationType']}")
        print(f"     目標: {rel['target']}")
    
    try:
        result = await memory_mcp.session.call_tool(
            "create_entities",
            {"entities": [trading_result]}
        )
        print(f"\n✅ 實體創建成功")
    except Exception as e:
        print(f"❌ 創建失敗: {e}")
        return
    
    # 第四步: 讀取完整知識圖譜
    print("\n\n4️⃣  讀取完整知識圖譜")
    print("-" * 80)
    
    try:
        result = await memory_mcp.session.call_tool(
            "read_graph",
            {}
        )
        
        if hasattr(result, "content") and result.content:
            content_item = result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            
            try:
                data = json.loads(text_content)
                entities = data.get("nodes", [])
                relations = data.get("relations", [])
                
                print(f"✅ 圖譜讀取成功")
                print(f"\n📊 圖譜統計:")
                print(f"  • 總實體數: {len(entities)}")
                print(f"  • 總關係數: {len(relations)}")
                
                # 顯示實體列表
                print(f"\n🔹 實體列表:")
                for i, entity in enumerate(entities, 1):
                    obs_count = len(entity.get('observations', []))
                    print(f"  {i}. {entity.get('name', 'N/A')}")
                    print(f"     類型: {entity.get('entityType', 'N/A')}")
                    print(f"     觀測數: {obs_count}")
                
                # 顯示關係列表
                print(f"\n🔗 關係列表:")
                for i, relation in enumerate(relations, 1):
                    print(f"  {i}. {relation.get('from', 'N/A')}")
                    print(f"     ─({relation.get('relationType', 'N/A')})→")
                    print(f"     {relation.get('to', 'N/A')}")
                
                # 繪製知識圖譜
                print(f"\n📈 知識圖譜可視化:")
                print(f"\n  市場分析")
                print(f"       ↑")
                print(f"  based_on")
                print(f"       ↑")
                print(f"   交易決策 ←──related_to── 交易結果")
                print(f"       ↑                    ↓")
                print(f"       └────executes────→ (執行決策)")
                
            except json.JSONDecodeError as e:
                print(f"❌ JSON 解析失敗: {e}")
    except Exception as e:
        print(f"❌ 讀取圖譜失敗: {e}")
    
    # 第五步: 演示搜尋
    print("\n\n5️⃣  演示搜尋功能")
    print("-" * 80)
    
    search_queries = [
        "TSMC 買入",
        "交易決策",
        "市場分析"
    ]
    
    for query in search_queries:
        print(f"\n搜尋: '{query}'")
        try:
            result = await memory_mcp.session.call_tool(
                "search_nodes",
                {"query": query, "limit": 10}
            )
            
            if hasattr(result, "content") and result.content:
                content_item = result.content[0]
                text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
                
                try:
                    data = json.loads(text_content)
                    found_nodes = data.get("nodes", [])
                    
                    if found_nodes:
                        print(f"  ✅ 找到 {len(found_nodes)} 個結果:")
                        for i, node in enumerate(found_nodes[:3], 1):
                            print(f"    {i}. {node.get('name')}")
                    else:
                        print(f"  ⚠️  無結果")
                        
                except json.JSONDecodeError:
                    print(f"  ❌ 解析失敗")
        except Exception as e:
            print(f"  ❌ 查詢失敗: {e}")
    
    print(f"\n✅ 演示完成")

# 執行演示
print("\n" + "=" * 80)
print("開始 Entity、Observation、Relation 關係演示")
print("=" * 80)

await demo_entity_observation_relation()


## 核心發現：search_nodes 查詢無結果的真正原因

基於官方源碼和實際測試的分析，問題確認如下：

### 問題分析

`search_nodes` 無法查到記錄的根本原因不是**查詢語句問題**，而是：

1. **search_nodes 的工作原理**（來自官方源碼）
   ```
   if 查詢是文本:
       1. 先嘗試為文本生成 embedding 向量
       2. 使用 vector_distance_cos() 進行向量相似度搜尋
       3. 如果向量搜尋有結果，返回（最多 5 個）
       4. 否則降級到文本搜尋：LIKE %query%
   else 查詢是向量:
       直接進行向量相似度搜尋
   ```

2. **search_nodes 的返回行為**
   - ✅ 返回格式：`{"nodes": [], "relations": []}`
   - ✅ 即使無結果也返回空數組，不會報錯
   - ⚠️ 最多返回 5 個結果（硬限制，不受 limit 參數影響）
   - ⚠️ 向量搜尋只搜索有 `embedding IS NOT NULL` 的實體

3. **你遇到的具體問題**
   ```
   create_entities: 說成功創建
   但 read_graph: 返回 0 個實體
   search_nodes: 因此無法找到任何記錄
   ```
   
   可能原因：
   - 🎯 **資料庫路徑問題**：每次初始化指向的可能是不同的 `.db` 文件
   - 或：MCP server 創建了內存中的實體，但沒有持久化到磁盤

### 驗證方法

```python
# 1. 檢查 read_graph 返回的實體數
result = await memory_mcp.session.call_tool("read_graph", {})
data = json.loads(result.content[0].text)
print(f"實體數: {len(data['nodes'])}")  # 應該 > 0

# 2. 如果是 0，確認 DB 路徑
print(memory_db_path)  # 確認指向的是預期的 .db 文件

# 3. 如果有實體，測試 search_nodes
result = await memory_mcp.session.call_tool(
    "search_nodes", 
    {"query": "你的查詢詞"}
)
data = json.loads(result.content[0].text)
print(f"找到: {len(data['nodes'])} 個結果")
```

### 解決方案

1. **確保資料庫路徑一致**
   - 檢查 `initialize_memory_mcp()` 中的 `LIBSQL_URL`
   - 確保每次都指向**同一個** `.db` 文件

2. **查詢需要匹配實體內容**
   - 文本搜尋使用 LIKE %query%，即 substring 匹配
   - 需要查詢词出現在實體的 `name`、`entityType` 或 `observations` 中

3. **查詢語句示例**
   ```python
   # ✅ 正確的查詢
   await memory_mcp.session.call_tool(
       "search_nodes",
       {"query": "TSMC"}  # 字符串查詢
   )
   
   # ✅ 也可以用向量
   embedding = [0.1, 0.2, 0.3, ...]  # embedding 數組
   await memory_mcp.session.call_tool(
       "search_nodes",
       {"query": embedding}  # 向量查詢
   )

In [ ]:
async def test_search_nodes_correct_usage():
    """
    演示 search_nodes 的正確使用方式
    
    基於官方源碼的理解：
    - 需要先創建實體
    - search_nodes 支持文本查詢和向量查詢
    - 返回最多 5 個結果（內部限制）
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 70)
    print("演示: search_nodes 正確用法")
    print("=" * 70)
    
    # 步驟 1: 創建測試實體
    print("\n1️⃣  創建測試實體")
    print("-" * 70)
    
    test_entities = [
        {
            "name": "trading_decision_2025_01_15",
            "entityType": "trading_decision",
            "observations": [
                "2025-01-15 10:30 買入決策",
                "TSMC 2330 目標價 600 元",
                "技術指標: RSI 40, MACD 正向",
                "成交量確認上升趨勢"
            ]
        },
        {
            "name": "market_analysis_2025_01_15",
            "entityType": "market_analysis",
            "observations": [
                "台灣股市開盤 17,500",
                "半導體板塊漲幅 2.3%",
                "TSMC 出現買入機會",
                "外資淨買超 150 億"
            ]
        },
        {
            "name": "trading_result_2025_01_14",
            "entityType": "trading_result",
            "observations": [
                "2025-01-14 完成交易",
                "買入 TSMC 1000 股",
                "成交價 615 元",
                "盈利 3000 元"
            ]
        }
    ]
    
    try:
        for entity in test_entities:
            result = await memory_mcp.session.call_tool(
                "create_entities",
                {"entities": [entity]}
            )
            print(f"  ✓ 創建實體: {entity['name']}")
    except Exception as e:
        print(f"  ✗ 創建實體失敗: {e}")
        return
    
    # 步驟 2: 測試文本查詢
    print("\n2️⃣  文本查詢示例")
    print("-" * 70)
    
    text_queries = [
        "TSMC 買入",
        "trading decision",
        "市場分析",
        "2025-01-15",
        "技術指標"
    ]
    
    for query in text_queries:
        print(f"\n  查詢: '{query}'")
        try:
            result = await memory_mcp.session.call_tool(
                "search_nodes",
                {
                    "query": query,
                    "limit": 10  # 實際上最多返回 5 個
                }
            )
            
            if hasattr(result, "content") and result.content:
                content_item = result.content[0]
                text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
                
                try:
                    data = json.loads(text_content)
                    found_nodes = data.get("nodes", [])
                    print(f"  結果: 找到 {len(found_nodes)} 個實體")
                    for i, node in enumerate(found_nodes[:3], 1):
                        print(f"    {i}. {node.get('name')} ({node.get('entityType')})")
                except json.JSONDecodeError:
                    print(f"  ✗ 解析回應失敗")
        except Exception as e:
            print(f"  ✗ 查詢失敗: {e}")
    
    # 步驟 3: 驗證完整圖表
    print("\n3️⃣  驗證保存結果")
    print("-" * 70)
    
    try:
        result = await memory_mcp.session.call_tool(
            "read_graph",
            {}
        )
        
        if hasattr(result, "content") and result.content:
            content_item = result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            
            try:
                data = json.loads(text_content)
                nodes = data.get("nodes", [])
                print(f"✓ 資料庫現在有 {len(nodes)} 個實體")
                print(f"\n  [實體列表]")
                for i, node in enumerate(nodes, 1):
                    print(f"  {i}. {node.get('name')} - {len(node.get('observations', []))} 個觀測")
            except json.JSONDecodeError:
                print(f"✗ 解析圖表失敗")
    except Exception as e:
        print(f"✗ 讀取圖表失敗: {e}")
    
    print(f"\n✅ 演示完成")

# 執行演示
print("\n\n" + "=" * 70)
print("現在執行正確用法演示")
print("=" * 70)

await test_search_nodes_correct_usage()


In [ ]:
async def test_search_nodes():
    """
    測試 search_nodes 工具 - 完整診斷版本
    
    根據官方源碼分析，search_nodes 的行為：
    1. 先嘗試向量搜尋（需要 embedding）
    2. 失敗則回退到文本搜尋（LIKE %query%）
    3. 最多返回 5 個結果
    
    查詢無結果的常見原因：
    - 資料庫為空（0 個實體）
    - 實體沒有 embedding 信息
    - 查詢關鍵字不匹配任何實體
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 70)
    print("診斷: search_nodes 查詢無結果")
    print("=" * 70)
    
    # 步驟 1: 檢查資料庫是否有數據
    print("\n🔍 步驟 1: 檢查資料庫中是否有實體")
    print("-" * 70)
    
    try:
        result_read = await memory_mcp.session.call_tool(
            "read_graph",
            {}
        )
        
        if hasattr(result_read, "content") and result_read.content:
            content_item = result_read.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            
            try:
                data = json.loads(text_content)
                nodes = data.get("nodes", [])
                
                print(f"✓ 資料庫連接正常")
                print(f"  • 當前實體數量: {len(nodes)}")
                
                if len(nodes) == 0:
                    print("\n⚠️  問題確認: 資料庫中沒有任何實體")
                    print("\n✅ 解決方案:")
                    print("  1. 先執行 create_entities 創建測試實體")
                    print("  2. 確保實體被正確保存")
                    print("  3. 然後再執行 search_nodes")
                    return None
                
                # 分析實體結構
                print(f"\n✓ 資料庫有 {len(nodes)} 個實體")
                print(f"\n  [實體結構分析]")
                sample_node = nodes[0]
                print(f"  • 節點字段: {list(sample_node.keys())}")
                print(f"  • 是否有 embedding: {'embedding' in sample_node and sample_node['embedding'] is not None}")
                
                # 統計有 embedding 的實體
                entities_with_embedding = sum(1 for n in nodes if 'embedding' in n and n['embedding'] is not None)
                print(f"  • 有 embedding 的實體: {entities_with_embedding}/{len(nodes)}")
                
                # 顯示所有實體
                print(f"\n  [所有實體列表]")
                for i, node in enumerate(nodes, 1):
                    has_emb = "有" if ('embedding' in node and node['embedding'] is not None) else "無"
                    print(f"    {i}. {node.get('name', 'N/A'):40} [{has_emb} embedding]")
                    
            except json.JSONDecodeError as e:
                print(f"✗ JSON 解析失敗: {e}")
                return None
        else:
            print("✗ 無法讀取圖表")
            return None
            
    except Exception as e:
        print(f"✗ 讀取圖表失敗: {type(e).__name__}: {str(e)}")
        return None
    
    # 步驟 2: 嘗試多種查詢策略
    print("\n\n🔍 步驟 2: 測試查詢策略")
    print("-" * 70)
    
    # 使用實際存在的實體名稱進行測試
    search_strategies = []
    if nodes:
        # 策略1: 完整的實體名稱
        full_name = nodes[0].get('name', '')
        search_strategies.append(("完整實體名稱", full_name))
        
        # 策略2: 實體名稱的前綴
        if len(full_name) > 5:
            prefix = full_name[:5]
            search_strategies.append(("名稱前綴", prefix))
        
        # 策略3: 使用 entityType
        entity_type = nodes[0].get('entityType', '')
        if entity_type:
            search_strategies.append(("實體類型", entity_type))
        
        # 策略4: 使用 observations
        observations = nodes[0].get('observations', [])
        if observations and len(observations) > 0:
            obs_word = observations[0].split()[0] if observations[0] else ""
            if obs_word:
                search_strategies.append(("觀測詞", obs_word))
    
    # 添加一些通用策略
    search_strategies.extend([
        ("空字符串", ""),
        ("簡單詞", "agent"),
    ])
    
    search_results = {}
    
    for strategy_name, query in search_strategies:
        print(f"\n  [{strategy_name}]")
        print(f"  查詢: '{query}'")
        
        try:
            result = await memory_mcp.session.call_tool(
                "search_nodes",
                {
                    "query": query,
                    "limit": 10,  # 注: 實際上最多返回 5 個
                }
            )
            
            if hasattr(result, "content") and result.content:
                content_item = result.content[0]
                text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
                
                try:
                    data = json.loads(text_content)
                    found_nodes = data.get("nodes", [])
                    print(f"  ✓ 找到 {len(found_nodes)} 個節點")
                    
                    search_results[strategy_name] = {
                        "query": query,
                        "count": len(found_nodes),
                        "nodes": found_nodes
                    }
                    
                    if found_nodes:
                        print(f"  [前 3 個結果]")
                        for i, node in enumerate(found_nodes[:3], 1):
                            print(f"    {i}. {node.get('name', 'N/A')}")
                    
                except json.JSONDecodeError as e:
                    print(f"  ✗ 回應解析失敗: {str(e)[:50]}")
            else:
                print(f"  ✗ 無有效回應")
                
        except Exception as e:
            print(f"  ✗ 查詢異常: {type(e).__name__}")
    
    # 步驟 3: 彙總診斷結果
    print("\n\n🔍 步驟 3: 診斷結論")
    print("-" * 70)
    
    successful_queries = {k: v for k, v in search_results.items() if v["count"] > 0}
    
    print(f"\n📊 查詢統計:")
    print(f"  • 總策略數: {len(search_results)}")
    print(f"  • 成功的查詢: {len(successful_queries)}")
    print(f"  • 失敗的查詢: {len(search_results) - len(successful_queries)}")
    
    if successful_queries:
        print(f"\n✅ 好消息: 某些查詢策略有效")
        for strategy, result in successful_queries.items():
            print(f"  ✓ {strategy}: 找到 {result['count']} 個節點")
    else:
        print(f"\n❌ 診斷: 所有查詢策略都失敗")
        print(f"\n可能的原因和解決方案:")
        print(f"  1. 資料庫中的實體可能沒有 embedding 信息")
        print(f"     → 檢查上方的 embedding 統計")
        print(f"  2. 文本搜尋使用 LIKE %query% 進行模糊匹配")
        print(f"     → 嘗試使用更通用的關鍵字")
        print(f"  3. 向量搜尋可能需要特定的 embedding 格式")
        print(f"     → 確保 embedding 是有效的數字陣列")
    
    print(f"\n✓ 診斷完成")
    return search_results

# 執行診斷測試
result_search_diag = await test_search_nodes()


## Section 6: 測試 read_graph 工具

In [ ]:
async def test_read_graph():
    """
    測試 read_graph 工具
    
    此工具用於讀取完整或部分的知識圖表。
    在 clear_old_memories 中使用，用於檢索所有節點以進行過濾和刪除。
    
    預期響應: 包含節點和關係的完整圖表
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 60)
    print("測試: read_graph")
    print("=" * 60)
    
    try:
        print(f"\n📤 發送請求:")
        print(f"   工具: read_graph")
        print(f"   讀取完整圖表")
        
        result = await memory_mcp.session.call_tool(
            "read_graph",
            {}
        )
        
        print(f"\n📥 收到回應:")
        print(f"   類型: {type(result)}")
        
        if hasattr(result, "content") and result.content:
            content_item = result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            
            print(f"   原始文本長度: {len(text_content)}")
            
            # 嘗試解析為 JSON
            try:
                data = json.loads(text_content)
                print(f"\n   ✓ JSON 解析成功")
                
                nodes = data.get("nodes", [])
                relations = data.get("relations", [])
                
                print(f"   節點總數: {len(nodes)}")
                print(f"   關係總數: {len(relations)}")
                
                # 顯示統計信息
                if nodes:
                    print(f"\n   [圖表統計]")
                    entity_types = set()
                    for node in nodes:
                        if "entityType" in node:
                            entity_types.add(node["entityType"])
                    
                    print(f"   實體類型: {entity_types}")
                    print(f"\n   [節點示例 (前 3 個)]")
                    for i, node in enumerate(nodes[:3]):
                        print(f"     {i+1}. {node.get('name', 'N/A')}")
                        print(f"        類型: {node.get('entityType', 'N/A')}")
                        print(f"        創建: {node.get('created_at', 'N/A')}")
                
            except json.JSONDecodeError as e:
                print(f"\n   ✗ JSON 解析失敗: {e}")
        
        print(f"\n✓ read_graph 測試完成")
        return result
    
    except Exception as e:
        print(f"\n✗ 錯誤: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 執行測試
result_read = await test_read_graph()


## Section 7: 測試 delete_entity 工具

In [ ]:
async def test_delete_entity():
    """
    測試 delete_entity 工具
    
    此工具用於刪除指定名稱的實體及其所有關聯。
    在 clear_old_memories 中使用，用於刪除過期的記憶體。
    
    預期響應: 成功/失敗消息
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 60)
    print("測試: delete_entity")
    print("=" * 60)
    
    try:
        # 先建立一個待刪除的實體
        test_entity_name = "agent_test_delete_2025"
        
        print(f"\n1️⃣  先建立測試實體進行刪除")
        create_result = await memory_mcp.session.call_tool(
            "create_entities",
            {
                "entities": [{
                    "name": test_entity_name,
                    "entityType": "test_deletion",
                    "observations": ["This entity will be deleted"]
                }]
            }
        )
        print(f"   ✓ 實體已建立: {test_entity_name}")
        
        # 現在刪除該實體
        print(f"\n2️⃣  刪除實體")
        print(f"   工具: delete_entity")
        print(f"   實體名稱: {test_entity_name}")
        
        delete_result = await memory_mcp.session.call_tool(
            "delete_entity",
            {"name": test_entity_name}
        )
        
        print(f"\n📥 收到回應:")
        print(f"   類型: {type(delete_result)}")
        
        if hasattr(delete_result, "content") and delete_result.content:
            content_item = delete_result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            print(f"   回應: {text_content[:200]}")
            
            if "success" in text_content.lower() or "deleted" in text_content.lower():
                print(f"   ✓ 刪除操作成功")
            else:
                print(f"   ⚠ 回應不明確，可能已刪除")
        
        print(f"\n✓ delete_entity 測試完成")
        return delete_result
    
    except Exception as e:
        print(f"\n✗ 錯誤: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 執行測試
result_delete = await test_delete_entity()


## Section 8: 驗證 memory_tools.py 中的函數邏輯

In [ ]:
from trading.tools.memory_tools import (
    load_execution_memory,
    save_execution_memory,
    recall_recent_decisions,
)

async def test_load_execution_memory():
    """
    驗證 load_execution_memory 函數
    
    此函數應該：
    1. 使用 search_nodes 查詢過去的決策
    2. 解析 JSON 響應
    3. 轉換為記憶體格式
    """
    global memory_mcp
    
    print("\n" + "=" * 60)
    print("驗證: load_execution_memory (來自 memory_tools.py)")
    print("=" * 60)
    
    try:
        agent_id = "test_123"
        
        # 先建立一些測試數據
        print(f"\n1️⃣  建立測試決策數據")
        await memory_mcp.session.call_tool(
            "create_entities",
            {
                "entities": [{
                    "name": f"agent_{agent_id}_decision_2025",
                    "entityType": "trading_decision",
                    "observations": [
                        "2025-01-10 10:30",
                        "買入決策",
                        "TSMC 2330 100股",
                        "技術指標看好"
                    ]
                }]
            }
        )
        print(f"   ✓ 測試決策已建立")
        
        # 現在測試 load_execution_memory
        print(f"\n2️⃣  調用 load_execution_memory")
        memory = await load_execution_memory(memory_mcp, agent_id)
        
        print(f"\n📊 結果:")
        print(f"   返回類型: {type(memory)}")
        print(f"   過去決策數量: {len(memory.get('past_decisions', []))}")
        
        if memory.get("past_decisions"):
            print(f"\n   [決策示例]")
            for i, decision in enumerate(memory["past_decisions"][:3]):
                print(f"     {i+1}. 日期: {decision.get('date', 'N/A')}")
                print(f"        動作: {decision.get('action', 'N/A')}")
                print(f"        原因: {decision.get('reason', 'N/A')}")
        
        print(f"\n✓ load_execution_memory 驗證完成")
        return memory
    
    except Exception as e:
        print(f"\n✗ 錯誤: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

async def test_save_execution_memory():
    """
    驗證 save_execution_memory 函數
    
    此函數應該：
    1. 準備執行結果摘要
    2. 使用 create_entities 保存到記憶體
    3. 返回成功/失敗狀態
    """
    global memory_mcp
    
    print("\n" + "=" * 60)
    print("驗證: save_execution_memory (來自 memory_tools.py)")
    print("=" * 60)
    
    try:
        agent_id = "test_456"
        execution_result = "買入 2330 100 股，成交價 620 元，總成本 62,000 元。交易成功。"
        mode = "AUTO_TRADING"
        
        print(f"\n1️⃣  保存執行結果")
        print(f"   Agent ID: {agent_id}")
        print(f"   結果: {execution_result[:60]}...")
        print(f"   模式: {mode}")
        
        success = await save_execution_memory(
            memory_mcp,
            agent_id,
            execution_result,
            mode=mode
        )
        
        print(f"\n✓ 保存結果: {'成功' if success else '失敗'}")
        return success
    
    except Exception as e:
        print(f"\n✗ 錯誤: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# 執行驗證測試
print("\n\n" + "=" * 60)
print("開始驗證 memory_tools.py 中的函數")
print("=" * 60)

memory_result = await test_load_execution_memory()
save_result = await test_save_execution_memory()


## Section 9: 總結與最佳實踐

### 關鍵發現

1. **call_tool 返回結構**
   - 返回對象包含 `content` 屬性（列表）
   - 每個 content 項目都是 TextContent，包含 `text` 屬性
   - 文本內容通常是 JSON 字符串，需要解析

2. **回應解析模式**
   ```python
   result = await memory_mcp.session.call_tool(tool_name, params)
   content_item = result.content[0]
   text_content = content_item.text
   data = json.loads(text_content)  # 解析 JSON
   ```

3. **memory_tools.py 中的三個關鍵工具**
   
   | 工具 | 用途 | 返回格式 |
   |-----|------|--------|
   | `create_entities` | 建立/更新實體 | JSON with creation status |
   | `search_nodes` | 查詢相似節點 | JSON with nodes array |
   | `read_graph` | 讀取完整圖表 | JSON with nodes and relations |
   | `delete_entity` | 刪除實體 | 成功/失敗消息 |

4. **錯誤處理建議**
   - 始終檢查 `hasattr(result, "content")` 和 `result.content`
   - 使用 try/except 捕捉 JSON 解析錯誤
   - 記錄原始回應用於調試

### 改進建議

針對 `memory_tools.py` 的改進點：

1. ✓ 當前解析邏輯正確
2. ✓ 錯誤處理適當
3. ✓ 日誌記錄充足
4. 📝 可考慮增加回應驗證層
5. 📝 可考慮建立回應模型（Pydantic）用於類型安全

## Section 10: 清理資源

## search_nodes 快速參考指南

### ✅ 正確的查詢方式

#### 1. 文本查詢（自動轉向量→向量搜尋→文本搜尋）
```python
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {
        "query": "TSMC",  # 字符串查詢
        "limit": 10  # 注意：實際最多返回 5 個
    }
)
```

#### 2. 向量查詢（直接向量相似度搜尋）
```python
# embedding 是一個數字陣列
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {
        "query": [0.1, 0.2, 0.3, ...],  # 向量陣列
        "limit": 10
    }
)
```

#### 3. 提取結果
```python
data = json.loads(result.content[0].text)
nodes = data.get("nodes", [])  # 搜索結果實體列表
relations = data.get("relations", [])  # 實體間的關係
```

### ❌ 常見錯誤

| 錯誤 | 原因 | 修復 |
|-----|------|------|
| 返回 `nodes: []` | 資料庫無實體或查詢不匹配 | 先 `create_entities`，確認 `read_graph` 有數據 |
| 返回超過 5 個結果 | 期望不符 | search_nodes 最多返回 5 個，這是源碼限制 |
| 使用 `query=""` 拋錯 | 不允許空字符串查詢 | 使用有效的查詢詞或向量陣列 |
| 收不到向量搜尋結果 | 實體沒有 embedding | create_entities 時需提供 `embedding` 字段 |

### 🔧 調試步驟

```python
# 1. 驗證資料庫有實體
result = await memory_mcp.session.call_tool("read_graph", {})
nodes = json.loads(result.content[0].text).get("nodes", [])
print(f"資料庫有 {len(nodes)} 個實體")  # 應 > 0

# 2. 檢查實體結構
if nodes:
    print(nodes[0].keys())  # 查看可用字段
    print(nodes[0])  # 查看完整內容

# 3. 嘗試文本查詢
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {"query": "你的關鍵詞"}  # 使用實體名稱的一部分
)
found = json.loads(result.content[0].text).get("nodes", [])
print(f"找到 {len(found)} 個結果")
```

### 📚 官方源碼參考

- Repository: https://github.com/joleyline/mcp-memory-libsql
- 核心文件: `src/services/graph-service.ts`
- API 定義: `src/index.ts` 中的 `ListToolsRequestSchema`

## Entity、Observation、Relation 快速參考

### 📊 核心概念速查表

| 組件 | 定義 | 例子 | 與DB的關係 |
|-----|------|------|----------|
| **Entity** | 知識圖的**節點**<br/>代表一個重要概念 | "trading_decision_2025_01_15" | 1 行在 `entities` 表 |
| **Observation** | Entity 的**屬性/事實** <br/>詳細描述信息 | "買入 1000 股 TSMC" | N 行在 `observations` 表<br/>外鍵指向 entity |
| **Relation** | Entity 間的**有向邊** <br/>表示邏輯關係 | "決策基於市場分析" | 1 行在 `relations` 表<br/>source→target |

### 🔑 工作流程

```
Step 1: 創建 Entity
│
├─ name: 唯一標識 (如 "market_analysis_2025_01_15")
├─ entityType: 分類 (如 "market_analysis")
├─ embedding: [向量]（用於語義搜尋）
└─ created_at: 時間戳

Step 2: 添加 Observations
│
├─ Obs 1: "台灣股市開盤 17,500"
├─ Obs 2: "半導體漲 2.3%"
└─ ... (可多個)

Step 3: 創建 Relations（指向其他 Entities）
│
└─ source (當前 entity)
   ─(relation_type: "based_on")→
   target (另一個 entity)

Step 4: 查詢
│
├─ read_graph(): 獲得完整知識圖
├─ search_nodes(): 基於查詢詞搜尋
└─ 結果包含 nodes 和 relations
```

### 💻 API 調用模式

#### 創建完整實體（包含觀測和關係）
```python
await memory_mcp.session.call_tool(
  "create_entities",
  {
    "entities": [{
      "name": "entity_name",
      "entityType": "type_name",
      "observations": [
        "obs1",
        "obs2",
        "obs3"
      ],
      "relations": [{
        "target": "other_entity_name",
        "relationType": "relation_type"
      }]
    }]
  }
)
```

#### 單獨創建關係
```python
await memory_mcp.session.call_tool(
  "create_relations",
  {
    "relations": [{
      "source": "entity1",
      "target": "entity2",
      "type": "relation_type"
    }]
  }
)
```

#### 讀取完整圖譜
```python
result = await memory_mcp.session.call_tool("read_graph", {})
data = json.loads(result.content[0].text)

nodes = data["nodes"]       # Entity 列表
relations = data["relations"]  # Relation 列表

# 構造圖譜
for relation in relations:
    from_entity = relation["from"]      # source entity name
    to_entity = relation["to"]          # target entity name
    rel_type = relation["relationType"]
    # from_entity -[rel_type]-> to_entity
```

### 🗄️ 數據庫查詢示例

如果你想直接查詢 SQLite 數據庫：

```sql
-- 獲取所有實體
SELECT * FROM entities;

-- 獲取某實體的所有觀測
SELECT content FROM observations 
WHERE entity_name = 'trading_decision_2025_01_15'
ORDER BY created_at;

-- 獲取某實體的所有出站關係
SELECT target, relation_type FROM relations
WHERE source = 'trading_decision_2025_01_15';

-- 獲取指向某實體的所有入站關係
SELECT source, relation_type FROM relations
WHERE target = 'trading_decision_2025_01_15';

-- 完整的知識圖譜（E-O-R）
SELECT 
  e.name,
  e.entity_type,
  o.content as observation,
  r.target as related_entity,
  r.relation_type
FROM entities e
LEFT JOIN observations o ON e.name = o.entity_name
LEFT JOIN relations r ON e.name = r.source
ORDER BY e.name, o.id, r.id;
```

### 🎯 設計原則

1. **原子性**：Entity 是最小單位，不能再分割
2. **靈活性**：Observations 可以任意增加，無需改變 schema
3. **關聯性**：Relation 定義實體間的邏輯，支持複雜業務模型
4. **可搜尋性**：Entity 有 embedding 用於語義搜尋，Observation 用於關鍵詞搜尋
5. **可追溯性**：所有組件都有 created_at 時間戳

### ❓ 常見設計問題

**Q: 何時該建立新 Entity vs 新 Observation?**
```
答: 
- 如果是獨立的概念/事件 → 新 Entity
- 如果是已知 Entity 的屬性/詳情 → 新 Observation

例:
✅ "TSMC股票" 和 "買入決策" → 兩個 Entity
✅ "TSMC股價615元" → "TSMC股票" 的 Observation
❌ "615元" 不單獨建 Entity（沒有獨立意義）
```

**Q: 何時用 Relation?**
```
答:
- 當你要表達實體間的邏輯關係時
- 用於查詢相關實體的網絡

例:
✅ 決策 -[based_on]-> 市場分析
✅ 結果 -[executes]-> 決策
❌ 不用 Relation 表達實體的屬性（用 Observation）
```

**Q: Embedding 放在 Entity 還是 Observation?**
```
答: 都可以，但通常:
- Entity embedding: 整個實體的語義向量（由系統自動生成或你提供）
- Observation: 存文本，沒有單獨的 embedding

當 search_nodes 時:
- 用 Entity 的 embedding 進行向量搜尋
- 用 Observation 的文本進行關鍵詞搜尋（LIKE）
```

## 按 entity_type 進行查詢

### 問題：能否直接指定 entity_type 過濾？

根據官方源碼分析，`search_nodes` **沒有直接的 `entity_type` 參數**，但可以通過查詢 entity_type 值間接實現：

### 官方源碼中的搜尋邏輯

```sql
-- 文本搜尋時，會在以下三個字段中搜尋：
WHERE e.name LIKE ? 
  OR e.entity_type LIKE ? 
  OR o.content LIKE ?
```

也就是說，查詢語句會搜索：
- `Entity.name`：實體的唯一名稱
- `Entity.entity_type`：實體的類型字段 ✅
- `Observation.content`：觀測內容

### 按 entity_type 查詢的方法

#### 方式 1：直接查詢 entity_type 名稱（推薦）

```python
# 查詢所有 "trading_decision" 類型的實體
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {
        "query": "trading_decision",  # 這會匹配 entity_type="trading_decision"
        "limit": 10
    }
)

data = json.loads(result.content[0].text)
entities = data["nodes"]
# 返回的實體會包括所有 entity_type 為 "trading_decision" 的實體
```

#### 方式 2：結合 entity_type 和其他信息

```python
# 查詢 "trading" 類型中包含 "TSMC" 的實體
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {
        "query": "trading_decision TSMC",  # 會匹配包含這些詞的實體
        "limit": 10
    }
)
```

#### 方式 3：在應用層進行過濾（精確匹配）

如果需要精確匹配 `entity_type`，可以在應用層過濾結果：

```python
async def search_by_entity_type(memory_mcp, entity_type: str, limit: int = 10):
    """
    按 entity_type 精確查詢
    """
    # 查詢 entity_type
    result = await memory_mcp.session.call_tool(
        "search_nodes",
        {
            "query": entity_type,
            "limit": limit
        }
    )
    
    data = json.loads(result.content[0].text)
    entities = data["nodes"]
    
    # 應用層過濾：只返回精確匹配的實體
    filtered = [e for e in entities if e.get("entityType") == entity_type]
    
    return {
        "nodes": filtered,
        "relations": data["relations"]
    }

# 使用
result = await search_by_entity_type(memory_mcp, "trading_decision")
```

### 實際例子

```python
# 假設已創建以下實體：
# 1. market_analysis_2025_01_15 (entityType: "market_analysis")
# 2. trading_decision_2025_01_15 (entityType: "trading_decision")
# 3. trading_result_2025_01_15 (entityType: "trading_result")

# 查詢所有交易決策
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {"query": "trading_decision"}
)

data = json.loads(result.content[0].text)
print(f"找到 {len(data['nodes'])} 個交易決策實體")

# 輸出示例：
# 找到 1 個交易決策實體
# {
#   "nodes": [
#     {
#       "name": "trading_decision_2025_01_15",
#       "entityType": "trading_decision",
#       "observations": [...],
#       ...
#     }
#   ]
# }
```

### 重要限制

| 限制項 | 說明 |
|-------|------|
| **最多 5 個結果** | 無論 `limit` 參數是多少，最多返回 5 個結果（硬限制） |
| **模糊匹配** | 使用 LIKE %query%，即子字符串匹配，不是精確匹配 |
| **搜索範圍** | 同時搜索 name、entity_type 和 observations |
| **無過濾器參數** | search_nodes 本身沒有 `filter` 或 `entityType` 參數 |

### 最佳實踐

1. **為 entity_type 使用有意義的名稱**
   ```python
   # ✅ 好的命名
   "entityType": "trading_decision"
   "entityType": "market_analysis"
   
   # ❌ 容易混淆的命名
   "entityType": "t1"
   "entityType": "m1"
   ```

2. **使用詳細的搜尋詞**
   ```python
   # ✅ 更容易找到
   await memory_mcp.session.call_tool(
       "search_nodes",
       {"query": "trading_decision TSMC 2330"}
   )
   
   # ⚠️ 可能與其他實體混淆
   await memory_mcp.session.call_tool(
       "search_nodes",
       {"query": "trading"}
   )
   ```

3. **在應用層維護索引**
   如果需要頻繁按 entity_type 查詢，建議在應用層維護一個索引：
   ```python
   # 初始化時構建索引
   async def build_entity_type_index(memory_mcp):
       result = await memory_mcp.session.call_tool("read_graph", {})
       data = json.loads(result.content[0].text)
       
       entity_type_index = {}
       for entity in data["nodes"]:
           entity_type = entity.get("entityType")
           if entity_type not in entity_type_index:
               entity_type_index[entity_type] = []
           entity_type_index[entity_type].append(entity.get("name"))
       
       return entity_type_index
   ```

In [ ]:
async def test_search_by_entity_type():
    """
    測試按 entity_type 進行查詢
    
    演示三種方式：
    1. 直接查詢 entity_type 名稱
    2. 結合 entity_type 和其他條件
    3. 應用層精確過濾
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 80)
    print("測試: 按 entity_type 進行查詢")
    print("=" * 80)
    
    # 準備測試數據
    print("\n1️⃣  創建測試實體")
    print("-" * 80)
    
    test_entities = [
        {
            "name": "market_analysis_tech_2025",
            "entityType": "market_analysis",
            "observations": ["技術指標看好", "半導體板塊上升"]
        },
        {
            "name": "market_analysis_finance_2025",
            "entityType": "market_analysis",
            "observations": ["融資余額下降", "資金面寬鬆"]
        },
        {
            "name": "trading_decision_buy_tsmc",
            "entityType": "trading_decision",
            "observations": ["買入 TSMC 2330", "數量 1000 股"]
        },
        {
            "name": "trading_decision_sell_amd",
            "entityType": "trading_decision",
            "observations": ["賣出 AMD", "獲利了結"]
        },
        {
            "name": "trading_result_success",
            "entityType": "trading_result",
            "observations": ["成交成功", "價格: 615 元"]
        }
    ]
    
    try:
        result = await memory_mcp.session.call_tool(
            "create_entities",
            {"entities": test_entities}
        )
        print(f"✅ 創建 {len(test_entities)} 個測試實體成功")
    except Exception as e:
        print(f"❌ 創建失敗: {e}")
        return
    
    # 方式 1: 直接查詢 entity_type
    print("\n\n2️⃣  方式 1: 直接查詢 entity_type 名稱")
    print("-" * 80)
    
    entity_types = ["market_analysis", "trading_decision", "trading_result"]
    
    for entity_type in entity_types:
        print(f"\n查詢: '{entity_type}'")
        try:
            result = await memory_mcp.session.call_tool(
                "search_nodes",
                {"query": entity_type, "limit": 10}
            )
            
            if hasattr(result, "content") and result.content:
                content_item = result.content[0]
                text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
                data = json.loads(text_content)
                nodes = data.get("nodes", [])
                
                print(f"  找到 {len(nodes)} 個結果:")
                for node in nodes:
                    print(f"    • {node.get('name')} (type: {node.get('entityType')})")
                    
        except Exception as e:
            print(f"  ❌ 查詢失敗: {e}")
    
    # 方式 2: 結合 entity_type 和其他條件
    print("\n\n3️⃣  方式 2: 結合 entity_type 和其他條件")
    print("-" * 80)
    
    combined_queries = [
        "trading_decision TSMC",
        "market_analysis 技術",
        "trading_result 成功"
    ]
    
    for query in combined_queries:
        print(f"\n查詢: '{query}'")
        try:
            result = await memory_mcp.session.call_tool(
                "search_nodes",
                {"query": query, "limit": 10}
            )
            
            if hasattr(result, "content") and result.content:
                content_item = result.content[0]
                text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
                data = json.loads(text_content)
                nodes = data.get("nodes", [])
                
                print(f"  找到 {len(nodes)} 個結果:")
                for node in nodes:
                    print(f"    • {node.get('name')}")
                    observations = node.get('observations', [])
                    if observations:
                        print(f"      觀測: {observations[0]}")
                        
        except Exception as e:
            print(f"  ❌ 查詢失敗: {e}")
    
    # 方式 3: 應用層精確過濾
    print("\n\n4️⃣  方式 3: 應用層精確過濾 (推薦用於精確匹配)")
    print("-" * 80)
    
    async def search_by_entity_type_exact(entity_type: str):
        """精確查詢指定 entity_type 的所有實體"""
        try:
            result = await memory_mcp.session.call_tool(
                "search_nodes",
                {"query": entity_type, "limit": 10}
            )
            
            if hasattr(result, "content") and result.content:
                content_item = result.content[0]
                text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
                data = json.loads(text_content)
                nodes = data.get("nodes", [])
                
                # 應用層過濾：只返回精確匹配的實體
                filtered = [n for n in nodes if n.get("entityType") == entity_type]
                
                return {
                    "query_entity_type": entity_type,
                    "total_found": len(nodes),
                    "exact_matched": len(filtered),
                    "nodes": filtered
                }
        except Exception as e:
            print(f"  ❌ 查詢失敗: {e}")
            return None
    
    print("\n精確查詢 'trading_decision' 類型的實體:")
    result = await search_by_entity_type_exact("trading_decision")
    if result:
        print(f"  查詢 entity_type: {result['query_entity_type']}")
        print(f"  搜尋結果總數: {result['total_found']}")
        print(f"  精確匹配數: {result['exact_matched']}")
        print(f"  結果:")
        for node in result['nodes']:
            print(f"    • {node.get('name')}")
    
    # 驗證
    print("\n\n5️⃣  驗證: 讀取完整圖譜確認")
    print("-" * 80)
    
    try:
        result = await memory_mcp.session.call_tool(
            "read_graph",
            {"limit": 100}
        )
        
        if hasattr(result, "content") and result.content:
            content_item = result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            data = json.loads(text_content)
            nodes = data.get("nodes", [])
            
            # 統計每個 entity_type 的數量
            entity_type_count = {}
            for node in nodes:
                entity_type = node.get("entityType")
                entity_type_count[entity_type] = entity_type_count.get(entity_type, 0) + 1
            
            print(f"✅ 圖譜中 entity_type 統計:")
            for entity_type, count in entity_type_count.items():
                print(f"  • {entity_type}: {count} 個實體")
                
    except Exception as e:
        print(f"❌ 讀取圖譜失敗: {e}")
    
    print(f"\n✅ 測試完成")

# 執行測試
print("\n開始測試按 entity_type 進行查詢")
await test_search_by_entity_type()


In [ ]:

# 診斷：檢查 DB 中的實際數據
import sqlite3

db_path = "/Users/sacahan/Documents/workspace/CasualTrader/backend/memory/test_memory.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("\n📊 數據庫中的實體:")
cursor.execute("SELECT name, entity_type FROM entities LIMIT 15")
for row in cursor.fetchall():
    print(f"  • {row[0]} (type: {row[1]})")

conn.close()

# 嘗試 read_graph
print("\n🔍 嘗試 read_graph:")
try:
    result = await memory_mcp.session.call_tool("read_graph", {"limit": 100})
    if hasattr(result, "content") and result.content:
        content_item = result.content[0]
        text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
        data = json.loads(text_content)
        print(f"  找到 {len(data.get('nodes', []))} 個實體")
except Exception as e:
    print(f"  ❌ 錯誤: {e}")


In [ ]:
async def cleanup():
    """
    清理 Memory MCP 連接和相關資源
    """
    global memory_mcp, _exit_stack
    
    print("\n" + "=" * 60)
    print("清理資源")
    print("=" * 60)
    
    try:
        if _exit_stack:
            await _exit_stack.aclose()
            print("✓ Exit stack 已關閉")
        
        memory_mcp = None
        print("✓ Memory MCP 連接已清理")
        
    except Exception as e:
        print(f"✗ 清理失敗: {type(e).__name__}: {str(e)}")

# 執行清理
await cleanup()
print("\n✓ Notebook 執行完成！")
